# Zurich Instruments LabOne Python API Example
## Run an AWG program using the command table

Demonstrate how to connect to a Zurich Instruments HDAWG and upload and run an
AWG program using the command table.

Requirements:
* LabOne Version >= 22.08
* Instruments:
    1 x HDAWG Instrument

---

In [ ]:
import textwrap
import json
import jsonschema
import numpy as np
import time

import zhinst.core
import zhinst.utils

Set up the connection. The connection is always through a session to a
Data Server. The Data Server then connects to the device.

The LabOne Data Server needs to run within the network, either on localhost when
starting LabOne on your local computer or a remote server.

In [ ]:
device_id = "dev8293"  # Device serial number available on its rear panel.
interface = "1GbE"  # For Ethernet connection.
# interface = "USB" # For all instruments connected to the host computer via USB.

server_host = "localhost"
server_port = 8004
api_level = 6  # Maximum API level supported for all instruments.

# Create an API session to the Data Server.
daq = zhinst.core.ziDAQServer(server_host, server_port, api_level)
# Establish a connection between Data Server and Device.
daq.connectDevice(device_id, interface)

# Create a base configuration: disable all available outputs, awgs,...
daq.syncSetInt(f"/{device_id}/system/preset/load", 1)
zhinst.utils.wait_for_state_change(daq, f"/{device_id:s}/system/preset/busy", 0, timeout = 2.0)

### Basic configuration
Specify the device configuration. Firstly select the AWG core, and therefore the output channels. Then set the output range for each channels and output gains. Finally, activate the outputs.

In [ ]:
awg_c = 0               # AWG core
output_range = 1.0      # Output range [V]

ch1 = 2*awg_c           # Output channel 1
ch2 = 2*awg_c+1         # Output channel 2

exp_setting = [
    [f"/{device_id:s}/system/awg/channelgrouping", 0],                  # Grouping mode 4x2 (HDAWG8) or 2x2 (HDAWG4)
    [f"/{device_id:s}/awgs/{awg_c:d}/outputs/0/gains/0", 1.0],          # Set the output gains matrix to diagonal
    [f"/{device_id:s}/awgs/{awg_c:d}/outputs/0/gains/1", 0.0],
    [f"/{device_id:s}/awgs/{awg_c:d}/outputs/1/gains/0", 0.0],
    [f"/{device_id:s}/awgs/{awg_c:d}/outputs/1/gains/1", 1.0],
    [f"/{device_id:s}/awgs/{awg_c:d}/outputs/0/modulation/mode", 0],    # Turn off modulation mode
    [f"/{device_id:s}/awgs/{awg_c:d}/outputs/1/modulation/mode", 0],
    [f"/{device_id:s}/sigouts/{ch1:d}/range", output_range],            # Select the output range
    [f"/{device_id:s}/sigouts/{ch2:d}/range", output_range],
    [f"/{device_id:s}/sigouts/{ch1:d}/on", True],                       # Turn on the outputs
    [f"/{device_id:s}/sigouts/{ch2:d}/on", True],
]
daq.set(exp_setting)

### AWG sequencer program
Define an AWG program as a string stored in the variable `awg_program`, equivalent to what would be entered in the Sequence Editor window in the graphical UI. Differently to a self-contained program, this example refers to a command table by the instruction `executeTableEntry`, and to a placeholder waveforms `p1` and `p2` by the instruction `placeholder`. Both the command table and the waveform data for the placeholders need to be uploaded separately before this sequence program can be run.

After defining the sequencer program, this must be compiled before being uploaded. Here the compilation is performed using the routine `compile_seqc` of the `zhinst.utils` package. The result of the compilation is a string in the binary elf format.

In [ ]:
# Define the AWG program
wfm_index = 10
wfm_length = 1024
awg_program = textwrap.dedent(
    f"""\
    // Define placeholder with length wfm_length
    wave p1 = placeholder({wfm_length:d});
    wave p2 = placeholder({wfm_length:d});

    // Assign an index to the placeholder waveform
    assignWaveIndex(1,p1, 2,p2, {wfm_index:d});

    while(true) {{
      executeTableEntry(0);
    }}
    """
)

# Compile the AWG program
device_type = daq.getString(f"/{device_id:s}/features/devtype")
options = daq.getString(f"/{device_id:s}/features/options")
samplerate = daq.getDouble(f"/{device_id:s}/system/clocks/sampleclock/freq")

elf, compiler_info = zhinst.core.compile_seqc(
    awg_program, devtype=device_type, options=options, index=awg_c, samplerate=samplerate
)

print(compiler_info)
assert not compiler_info[
    "messages"
], f"There was an error during compilation: {compiler_info['messages']:s}"

Upload the compiled sequence to the device.

In [ ]:
daq.setVector(f"/{device_id:s}/awgs/{awg_c:d}/elf/data", elf)

# Wait until the sequence is correctly uploaded
timeout = 10.0
start = time.time()
while daq.getInt(f"/{device_id:s}/awgs/{awg_c:d}/ready") == 0:
    if time.time() - start > timeout:
        raise TimeoutError(
            f"Sequence not uploaded within {timeout:.1f}s."
        )
    time.sleep(0.01)
print("Sequence successfully uploaded.")

### Command Table definition and upload

The waveforms is played by a command table, whose structure must conform to
a defined schema. The schema can be 
read from the device. This example validates the command table against the 
schema before uploading it. This step is not mandatory since the device will
validate the schema as well. However, it is helpful for debugging.

Read the schema from the device.

In [ ]:
schema_node = f"/{device_id:s}/awgs/0/commandtable/schema"
schema = json.loads(
    daq.get(schema_node, flat=True)[schema_node][0]["vector"]
)
print(f"The device is using the commandtable schema version {schema['version']}")

Define the command table and validate it against the schema.

In [ ]:
ct = {
    "header": {
        "version": "1.1.0",
    },
    "table": [
        {
            "index": 0,
            "waveform": {"index": wfm_index},
            "amplitude0": {"value": 1.0},
            "amplitude1": {"value": 1.0},
        }
    ],
}

jsonschema.validate(
    instance=ct,
    schema=schema,
    cls=jsonschema.Draft7Validator,
)

Upload the command table to the device.

In [ ]:
daq.set(f"/{device_id:s}/awgs/{awg_c:d}/commandtable/data", json.dumps(ct, separators=(",", ":")))

# Wait until the command table is correctly uploaded
timeout = 10.0
start = time.time()
while (True):
    status = daq.getInt(f"/{device_id:s}/awgs/{awg_c:d}/commandtable/status")

    if status & 0b1:
        # Upload successful
        break

    if status & 0b1000:
        # Error in command table
        raise RuntimeError(f"The upload of command table on core {awg_c:d} failed.")

    if time.time() - start >= timeout:
        # Timeout
        raise TimeoutError(f"Command table not uploaded within {timeout:.1f}s.")
    
    time.sleep(0.01)

print("Command tables upload successful")

### Waveform upload

Replace the placeholder waveform with a drag pulse (I quadrature is a gaussian and Q quadrature is the derivative of I). The waveform data is uploaded to the index `wfm_index`, which must be the same specified by
the `assignWaveIndex` sequencer instruction.

Define the waveform.

In [ ]:
x_array = np.linspace(-wfm_length//2, wfm_length//2, wfm_length)
sigma = wfm_length//8
# Define the waveform as a numpy array
waveform1 = np.array(
    np.exp(-np.power(x_array, 2.0) / (2 * np.power(sigma, 2.0))),
    dtype=float,
)
waveform2 = np.array(
    -x_array/sigma**2 * waveform1,
    dtype=float,
)

# Convert the numpy array to the native AWG waveform format
waveform_native = zhinst.utils.convert_awg_waveform(waveform1, -waveform2)

Upload the waveform to the device.

In [ ]:
daq.set(f"/{device_id:s}/awgs/{awg_c:d}/waveform/waves/{wfm_index:d}", waveform_native)

### Enable the AWG 
This is the preferred method of using the AWG: run in single mode. Continuous waveform playback
is best achieved by using an infinite loop (e.g., `while (true)`) in the sequencer program.

In [ ]:
daq.set(f"/{device_id:s}/awgs/{awg_c:d}/single", True)
daq.syncSetInt(f"/{device_id:s}/awgs/{awg_c:d}/enable", True)